1. Extracción y revisión de tdf_tours_master.csv

In [1]:
import pandas as pd 
import numpy as np
import re

df_tours = pd.read_csv('../data/raw/tdf_tours_master.csv')

display(df_tours)
display(df_tours.info())
display(df_tours.nunique())

,Year,Dates,Stages,Distance,Starters,Finishers
0,1903,119 July 1903,6,"2,428 km (1,509 mi)",60,21
1,1904,224 July 1904,6,"2,428 km (1,509 mi)",88,15
2,1905,930 July 1905,11,"2,994 km (1,860 mi)",60,24
3,1906,429 July 1906,13,"4,637 km (2,881 mi)",82,14
4,1907,8 July  4 August 1907,14,"4,488 km (2,789 mi)",93,33
...,...,...,...,...,...,...
107,2021,26 June  18 July 2021,21,"3,414.4 km (2,121.6 mi)",184,141
108,2022,124 July 2022,21,"3,328 km (2,068 mi)",176,135
109,2023,1–23 July 2023,21,"3,405.6 km (2,116 mi)",176,150
110,2024,29 June  21 July 2024,21,3498 km,176,141


<class 'pandas.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Year       112 non-null    int64
 1   Dates      112 non-null    str  
 2   Stages     112 non-null    str  
 3   Distance   112 non-null    str  
 4   Starters   112 non-null    int64
 5   Finishers  112 non-null    int64
dtypes: int64(3), str(3)
memory usage: 5.4 KB


None

Year         112
Dates        112
Stages        39
Distance     109
Starters      48
Finishers     77
dtype: int64

2. Limpiamos y transformamos la columna 'Distance'

In [2]:
#Vemos que la columna 'distance' tiene el formato 2,160.4 km (99.7 mi)
#Eliminamos las millas y los caracteres km y convertimos.

def extract_km (distance_str):
    km_index = distance_str.find ('km')
    km_value = distance_str[:km_index].strip()
    km_value = km_value.replace(',', '')

    return float (km_value)

df_tours['distance_in_km'] = df_tours['Distance'].apply(extract_km)
df_tours.drop(columns = 'Distance', inplace = True)

# display (df_tours)

# Compruebo que no hay valores nulos
# display(df_tours['distance_in_km'].isnull().sum())

#Elimino la columna 'Distance'
display (df_tours)

,Year,Dates,Stages,Starters,Finishers,distance_in_km
0,1903,119 July 1903,6,60,21,2428.0
1,1904,224 July 1904,6,88,15,2428.0
2,1905,930 July 1905,11,60,24,2994.0
3,1906,429 July 1906,13,82,14,4637.0
4,1907,8 July  4 August 1907,14,93,33,4488.0
...,...,...,...,...,...,...
107,2021,26 June  18 July 2021,21,184,141,3414.4
108,2022,124 July 2022,21,176,135,3328.0
109,2023,1–23 July 2023,21,176,150,3405.6
110,2024,29 June  21 July 2024,21,176,141,3498.0


3. Limpieza de la columna Stages

In [3]:
#Separo en dos la columna Stages. 
#Dejo dentro de la columna el número de etapas 
#y en una nueva columna el texto

# 1. Limpieza de seguridad: quitamos espacios invisibles (como \xa0) de toda la columna
# Esto es vital para que el número al principio se detecte bien
df_tours['Stages'] = df_tours['Stages'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()

# 2. Extracción con una Regex más flexible
# ^(\d+) -> Captura el número inicial
# \s* -> Salta cualquier espacio opcional que haya justo después del número
# (.*)   -> Captura TODO lo que quede hasta el final
df_tours[['Stages_clean', 'stages_observations']] = df_tours['Stages'].str.extract(r'^(\d+)\s*(.*)')

# 3. Mantenemos el número en la columna original y limpiamos la nueva
df_tours['Stages'] = df_tours['Stages_clean']
df_tours['stages_observations'] = df_tours['stages_observations'].str.strip()

# 4. Borramos la columna temporal
df_tours.drop(columns=['Stages_clean'], inplace=True)

#5. Convertimos a null las que no tiene texto en 'stages_observation'
df_tours['stages_observations'] = df_tours['stages_observations'].replace('', np.nan)

# display (df_tours[df_tours['Year'] == 1977])
#Quiero mostrar solo las lineas que no sean nulas en la columna stages_observations
# display (df_tours['stages_observations'].isna().sum())
# display (df_tours['stages_observations'].unique())

#Ahora, según lo que contenga la taba 'stages_observations', se suma a 'Stages' para que de el número correcto de stages.
add_stages = {np.nan: 0, ', including one split stage': 1, ', including six split stages': 6, ', including five split stages': 5, ', including eight split stages': 8,
              ', including one split stages': 1, ', including two split stages': 2, ', including three split stages': 3, '+ Prologue, including two split stages': 3,
              ', including four split stages': 4, '+ Prologue, including three split stages': 4, '+ Prologue, including five split stages': 6,
              '+ Prologue, including four split stages': 5, '+ Prologue, including six split stages': 7, '+ Prologue': 1, '+ Prologue, including one split stage': 2,
              '+ prologue': 1}

extra_stages = df_tours['stages_observations'].map(add_stages).astype(int)
df_tours['Stages'] = df_tours['Stages'].astype(int) + extra_stages

# display(df_tours[df_tours['Year'] == 1977])

4. Creo nueva columna con el ratio de abandonos

In [4]:
#Creo nueva columna con el % de abandonos
df_tours['abandon_rate'] = (1 - (df_tours['Finishers'] / df_tours ['Starters'])).round(2)
display(df_tours)

,Year,Dates,Stages,Starters,Finishers,distance_in_km,stages_observations,abandon_rate
0,1903,119 July 1903,6,60,21,2428.0,NaN,0.65
1,1904,224 July 1904,6,88,15,2428.0,NaN,0.83
2,1905,930 July 1905,11,60,24,2994.0,NaN,0.60
3,1906,429 July 1906,13,82,14,4637.0,NaN,0.83
4,1907,8 July  4 August 1907,14,93,33,4488.0,NaN,0.65
...,...,...,...,...,...,...,...,...
107,2021,26 June  18 July 2021,21,184,141,3414.4,NaN,0.23
108,2022,124 July 2022,21,176,135,3328.0,NaN,0.23
109,2023,1–23 July 2023,21,176,150,3405.6,NaN,0.15
110,2024,29 June  21 July 2024,21,176,141,3498.0,NaN,0.20


5. Limipiamos columna Date

In [5]:
def normalizar_fechas_final(fecha_texto):
    meses = {
        'January': '01', 'February': '02', 'March': '03', 'April': '04', 
        'May': '05', 'June': '06', 'July': '07', 'August': '08', 
        'September': '09', 'October': '10', 'November': '11', 'December': '12'
    }
    
    texto = str(fecha_texto)
    
    #Limpieza base
    texto = re.sub(r'\[.*?\]', '', texto)          
    texto = re.sub(r'\d{4}$', '', texto)           
    texto = texto.replace('\xa0', ' ')             
    
    # Reemplazamos CUALQUIER carácter que no sea alfanumérico (\w) o espacio (\s) por un '-'
    # Esto cazará automáticamente esos "–" o "—" invisibles y corruptos.
    texto = re.sub(r'[^\w\s]', '-', texto)
    
    # Limpiamos espacios alrededor del nuevo guion y colapsamos espacios dobles
    texto = re.sub(r'\s*-\s*', '-', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()     
    
    # Separación
    partes = texto.split('-')
    
    if len(partes) != 2:
        return fecha_texto 
        
    inicio_str = partes[0].strip()
    fin_str = partes[1].strip()
    
    inicio_partes = inicio_str.split()
    fin_partes = fin_str.split()
    
    try:
        # Lógica A: Mismo mes (ej. "1", "19 July")
        if len(inicio_partes) == 1:
            dia_inicio = inicio_partes[0].zfill(2)
            dia_fin = fin_partes[0].zfill(2)
            mes = meses.get(fin_partes[1], '00')
            return f"De {dia_inicio}/{mes} a {dia_fin}/{mes}"
            
        # Lógica B: Meses distintos (ej. "8 July", "4 August")
        else:
            dia_inicio = inicio_partes[0].zfill(2)
            mes_inicio = meses.get(inicio_partes[1], '00')
            dia_fin = fin_partes[0].zfill(2)
            mes_fin = meses.get(fin_partes[1], '00')
            return f"De {dia_inicio}/{mes_inicio} a {dia_fin}/{mes_fin}"
            
    except Exception as e:
        return fecha_texto

# Aplicar
df_tours['Dates'] = df_tours['Dates'].apply(normalizar_fechas_final)

# Mostrar resultados
display(df_tours[['Year', 'Dates']].head(10))
display(df_tours)

,Year,Dates
0,1903,De 01/07 a 19/07
1,1904,De 02/07 a 24/07
2,1905,De 09/07 a 30/07
3,1906,De 04/07 a 29/07
4,1907,De 08/07 a 04/08
5,1908,De 13/07 a 09/08
6,1909,De 05/07 a 01/08
7,1910,De 03/07 a 31/07
8,1911,De 02/07 a 30/07
9,1912,De 30/06 a 28/07


,Year,Dates,Stages,Starters,Finishers,distance_in_km,stages_observations,abandon_rate
0,1903,De 01/07 a 19/07,6,60,21,2428.0,NaN,0.65
1,1904,De 02/07 a 24/07,6,88,15,2428.0,NaN,0.83
2,1905,De 09/07 a 30/07,11,60,24,2994.0,NaN,0.60
3,1906,De 04/07 a 29/07,13,82,14,4637.0,NaN,0.83
4,1907,De 08/07 a 04/08,14,93,33,4488.0,NaN,0.65
...,...,...,...,...,...,...,...,...
107,2021,De 26/06 a 18/07,21,184,141,3414.4,NaN,0.23
108,2022,De 01/07 a 24/07,21,176,135,3328.0,NaN,0.23
109,2023,De 01/07 a 23/07,21,176,150,3405.6,NaN,0.15
110,2024,De 29/06 a 21/07,21,176,141,3498.0,NaN,0.20


6. Últimos cambios y guardado del DataFrame

In [6]:
#Ponemos todos los tituls de columnas en minusculas
df_tours.columns = df_tours.columns.str.lower()


#Ordenamos columnas
column_order = ['year', 'dates', 'stages', 'stages_observations', 'distance_in_km', 'starters', 'finishers', 'abandon_rate']
df_clean_tours = df_tours[column_order]

display(df_clean_tours)

df_clean_tours.to_csv('../data/processed/tdf_tours_clean.csv', index=False)

,year,dates,stages,stages_observations,distance_in_km,starters,finishers,abandon_rate
0,1903,De 01/07 a 19/07,6,NaN,2428.0,60,21,0.65
1,1904,De 02/07 a 24/07,6,NaN,2428.0,88,15,0.83
2,1905,De 09/07 a 30/07,11,NaN,2994.0,60,24,0.60
3,1906,De 04/07 a 29/07,13,NaN,4637.0,82,14,0.83
4,1907,De 08/07 a 04/08,14,NaN,4488.0,93,33,0.65
...,...,...,...,...,...,...,...,...
107,2021,De 26/06 a 18/07,21,NaN,3414.4,184,141,0.23
108,2022,De 01/07 a 24/07,21,NaN,3328.0,176,135,0.23
109,2023,De 01/07 a 23/07,21,NaN,3405.6,176,150,0.15
110,2024,De 29/06 a 21/07,21,NaN,3498.0,176,141,0.20
